In [0]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
sns.set(color_codes=True)
%matplotlib inline

df3 = pd.read_csv('train.csv')
df3.head()

1KMwq1AHuIi0M7t0oWedvLoUJNKESjnHD


,Id,category,text
0,1,snack,"""RecipeName"": "" Reese's™ Peanut Butter Layer B..."
1,2,dessert,"""RecipeName"": """"""""Just Bugging You"" Cupcake Co..."
2,3,dessert,"""RecipeName"": """"""Easy Fruit Crisp ""Dump"" Desse..."
3,4,snack,"""RecipeName"": ""3-Ingredient Crunchy Snack Mix""..."
4,5,condiment,"""RecipeName"": ""5-Minute Poppy Seed Dressing"",\..."


In [0]:
df3 = pd.read_csv('train.csv')

In [0]:
for i in range(0,len(df3)):
  x=df3.text[i]
  y=x.replace('\"','')
  t=y.index('[')+1
  t1=y.index(']')
  z=y.replace(y[:t],"")
  z=z.replace(y[t1:],"")
  z=z.replace("\r","")
  z=z.replace("\n","")
  df3.text[i]=""
  df3.text[i]=z

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [0]:
print(df3)

      Id   category                                               text
0      1      snack  pouch (17.5 oz) Betty Crocker™ double chocolat...
1      2    dessert  box Betty Crocker™ Super Moist™ cake mix (any ...
2      3    dessert  box Betty Crocker™ Super Moist™ yellow cake mi...
3      4      snack  cups popped light butter-flavor microwave popc...
4      5  condiment  cup mayonnaise or salad dressing,cup sugar,tab...
..   ...        ...                                                ...
740  741  appetizer  sheets frozen phyllo (filo) pastry (17x12 inch...
741  742    dessert  cups Gold Medal™ all-purpose flour,teaspoons b...
742  743  condiment  cups plain regular or fat-free yogurt (1 pound...
743  744    dessert  cups granulated sugar,cup vegetable oil,teaspo...
744  745  appetizer  cup mayonnaise,tablespoons refrigerated honey ...

[745 rows x 3 columns]


In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


In [0]:
print(df3)

      Id   category                                               text
0      1      snack  pouch (17.5 oz) Betty Crocker™ double chocolat...
1      2    dessert  box Betty Crocker™ Super Moist™ cake mix (any ...
2      3    dessert  box Betty Crocker™ Super Moist™ yellow cake mi...
3      4      snack  cups popped light butter-flavor microwave popc...
4      5  condiment  cup mayonnaise or salad dressing,cup sugar,tab...
..   ...        ...                                                ...
740  741  appetizer  sheets frozen phyllo (filo) pastry (17x12 inch...
741  742    dessert  cups Gold Medal™ all-purpose flour,teaspoons b...
742  743  condiment  cups plain regular or fat-free yogurt (1 pound...
743  744    dessert  cups granulated sugar,cup vegetable oil,teaspo...
744  745  appetizer  cup mayonnaise,tablespoons refrigerated honey ...

[745 rows x 3 columns]


In [0]:
import re
df3['text'] = df3['text'].apply(lambda x: re.sub(r'\b\d+\b', '', x))

df3['text']  = df3['text'] .apply(lambda x: re.sub(r'[^\w\s]','',x))

df3['text']  = df3['text'] .apply(lambda x: x.lower())

In [0]:
df3.head()

,Id,category,text
0,1,snack,pouch oz betty crocker double chocolate chunk...
1,2,dessert,box betty crocker super moist cake mix any fla...
2,3,dessert,box betty crocker super moist yellow cake mixc...
3,4,snack,cups popped light butterflavor microwave popco...
4,5,condiment,cup mayonnaise or salad dressingcup sugartable...


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorize = TfidfVectorizer()
X = vectorize.fit_transform(df3['text'])

print(X.shape)

(745, 3156)


,Weight
or,1.669305
oz,2.117557
chopped,2.176646
cups,2.286849
cup,2.296606


In [0]:
from sklearn.preprocessing import LabelEncoder
labeler = LabelEncoder()
labeler.fit(df3['category'])
df3['ml']=labeler.transform(df3['category'])
labeler.classes_

array(['appetizer', 'beverage', 'condiment', 'dessert', 'snack'],
      dtype=object)

In [0]:
df3['ml'].value_counts()

4    155
3    151
2    149
0    146
1    144
Name: ml, dtype: int64

In [0]:
y = df3['ml'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 7)
target_names = labeler.classes_

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
knn = KNeighborsClassifier(n_neighbors = 10, weights='uniform', algorithm='auto')
knn.fit(X_train, y_train) 
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred, target_names = target_names))
list1 = pd.DataFrame(labeler.classes_, columns = ['category'])

              precision    recall  f1-score   support

   appetizer       0.78      0.86      0.82        29
    beverage       0.86      0.76      0.81        25
   condiment       0.69      0.58      0.63        31
     dessert       0.71      0.86      0.77        28
       snack       0.77      0.75      0.76        36

    accuracy                           0.76       149
   macro avg       0.76      0.76      0.76       149
weighted avg       0.76      0.76      0.76       149



In [0]:
from sklearn.linear_model import LogisticRegression
logit= LogisticRegression()
logit.fit(X_train, y_train)
y_pred = logit.predict(X_test)
print(classification_report(y_test, y_pred,target_names = target_names))

              precision    recall  f1-score   support

   appetizer       0.84      0.90      0.87        29
    beverage       0.93      1.00      0.96        25
   condiment       0.78      0.68      0.72        31
     dessert       0.81      0.89      0.85        28
       snack       0.85      0.78      0.81        36

    accuracy                           0.84       149
   macro avg       0.84      0.85      0.84       149
weighted avg       0.84      0.84      0.84       149



In [0]:
def ingre(top):
    
    df = pd.DataFrame({ 'ingred':top})
    
    d = list1.to_dict()
    d= d['category']
    df["ingred"] = df["ingred"].map(d) 
    return df.to_string(header=False)
def predict_cate(ingredients):

    X_ing = vectorize.transform(ingredients)
   
    prob = logit.predict_proba(X_ing)[0]
   
    top = np.argsort(prob)[::-1][:1]
   
    top_prob = np.sort(prob)[::-1][:1]
   
    return ingre(top)

In [0]:
predict_cate(['pouch  oz betty crocker double chocolate chunk cookie mixoil water and egg called for on cookie mix pouchcup from oz can sweetened condensed milk not evaporatedcup creamy peanut buttercup from oz package reeses minis peanut butter cups coarsely choppedcup semisweet chocolate chipscup reeses peanut butter chipscup cocktail peanuts chopped'])

'0  snack'

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link='https://drive.google.com/open?id=1bf1xLhtgojK3-mLh4JxX-fIvlV3pEU1D'
fluff, id = link.split('=')
print (id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('validation.csv')  
df3 = pd.read_csv('validation.csv')
df3.head()

1bf1xLhtgojK3-mLh4JxX-fIvlV3pEU1D


,Id,text
0,746,"""RecipeName"": """"""Cinnamon ""Fried"" Ice Cream """"..."
1,747,"""RecipeName"": ""“Devilicious” Eggs"",\r\n""Ingred..."
2,748,"""RecipeName"": ""All Shook Up Chex Mix"",\r\n""Ing..."
3,749,"""RecipeName"": ""Annie’s™ Yogurt Dirt Cups "",\r\..."
4,750,"""RecipeName"": ""Antipasto Kabobs"",\r\n""Ingredie..."


In [0]:
df4=pd.read_csv('validation.csv')
for i in range(0,len(df4)):
  x=df4.text[i]
  y=x.replace('\"','')
  t=y.index('[')+1
  t1=y.index(']')
  z=y.replace(y[:t],"")
  z=z.replace(y[t1:],"")
  z=z.replace("\r","")
  z=z.replace("\n","")
  df4.text[i]=""
  df4.text[i]=z
import re
df4['text'] = df4['text'].apply(lambda x: re.sub(r'\b\d+\b', '', x))

df4['text']  = df4['text'] .apply(lambda x: re.sub(r'[^\w\s]','',x))

df4['text']  = df4['text'] .apply(lambda x: x.lower())


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [0]:
for i in range(len(df4)):
  print(df4.Id[i],predict_cate([df4.text[i]]))

746 0  beverage
747 0  appetizer
748 0  snack
749 0  snack
750 0  appetizer
751 0  beverage
752 0  appetizer
753 0  appetizer
754 0  appetizer
755 0  appetizer
756 0  beverage
757 0  dessert
758 0  dessert
759 0  appetizer
760 0  beverage
761 0  condiment
762 0  beverage
763 0  dessert
764 0  dessert
765 0  beverage
766 0  condiment
767 0  beverage
768 0  appetizer
769 0  appetizer
770 0  appetizer
771 0  dessert
772 0  snack
773 0  beverage
774 0  appetizer
775 0  dessert
776 0  beverage
777 0  appetizer
778 0  dessert
779 0  dessert
780 0  dessert
781 0  condiment
782 0  appetizer
783 0  beverage
784 0  dessert
785 0  appetizer
786 0  dessert
787 0  dessert
788 0  condiment
789 0  appetizer
790 0  beverage
791 0  snack
792 0  dessert
793 0  dessert
794 0  appetizer
795 0  appetizer
796 0  dessert
797 0  snack
798 0  condiment
799 0  snack
800 0  snack
801 0  snack
802 0  snack
803 0  dessert
804 0  dessert
805 0  dessert
806 0  snack
807 0  snack
808 0  snack
809 0  dessert
810 0  de

In [0]:
import csv

In [0]:
df4.to_csv(r'C:\Users\pawan\Downloads\submission.csv')